# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders= pd.read_csv("orders.csv")
display(orders)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [19]:
# your code here
vipc = orders.groupby(['CustomerID'])['amount_spent'].agg('sum')

vipc = vipc[vipc > vipc.quantile(q=0.95)]

len(vipc.index)

217

In [4]:
preferredc = orders.groupby(['CustomerID'])['amount_spent'].agg('sum')

preferredc = preferredc[(preferredc < preferredc.quantile(q=0.95)) & (preferredc > preferredc.quantile(q=0.75))]

preferredc.index

Int64Index([12347, 12348, 12349, 12352, 12356, 12360, 12362, 12370, 12371,
            12378,
            ...
            18241, 18242, 18245, 18251, 18257, 18259, 18260, 18272, 18283,
            18287],
           dtype='int64', name='CustomerID', length=868)

In [15]:
orders['c_label'] = orders['CustomerID'].apply((lambda x: "VIP" if x in list(vipc.index) else ("Preferred" if x in list(preferredc.index) else "-")))
orders

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,c_label
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,Preferred
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,Preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20,-
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60,-
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60,-
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60,-


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [28]:
# your code here
orders[orders['c_label'] == 'VIP'].groupby(['Country'])['CustomerID'].agg('nunique').sort_values(ascending=False)

Country
United Kingdom     177
Germany             10
France               9
Switzerland          3
Spain                2
Portugal             2
Australia            2
EIRE                 2
Japan                2
Finland              1
Belgium              1
Netherlands          1
Norway               1
Denmark              1
Singapore            1
Cyprus               1
Sweden               1
Channel Islands      1
Name: CustomerID, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [31]:
# your code here

orders[(orders['c_label'] == 'VIP') | (orders['c_label'] == 'Preferred')].groupby(['Country'])['CustomerID'].agg('nunique').sort_values(ascending=False)

Country
United Kingdom     932
Germany             39
France              29
Belgium             12
Switzerland          9
Spain                9
Portugal             7
Norway               7
Italy                5
Finland              5
Japan                4
Australia            4
Cyprus               4
Channel Islands      4
Austria              3
EIRE                 3
Denmark              3
Sweden               2
Israel               2
Iceland              1
Greece               1
Lebanon              1
Malta                1
Netherlands          1
Poland               1
Singapore            1
Canada               1
Name: CustomerID, dtype: int64